In [1]:
! pip install datasets
! pip install transformers
! pip install cleantext
! pip uninstall -y transformers accelerate
! pip install transformers accelerate
! pip install ipywidgetsb

/bin/bash: /home/oscar/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/oscar/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/oscar/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/oscar/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
/bin/bash: /home/oscar/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)


  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
/bin/bash: /home/oscar/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ERROR: Could not find a version that satisfies the requirement ipywidgetsb (from versions: none)
ERROR: No matching distribution found for ipywidgetsb


In [2]:
! export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:100'

/bin/bash: /home/oscar/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [3]:
import re
import os
import sys
import cleantext
import numpy as np
import pandas as pd
from cleantext import clean
from datasets import load_metric
from transformers import Trainer
from datasets import load_dataset
from transformers import  pipeline
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from pandas.core.arrays.integer import Int64Dtype
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [4]:
# in Google collab
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_data(example):
    max_length = 512
    tok = tokenizer(example['text'],
                    truncation=True,
                    max_length=max_length,
                    return_tensors='pt',
                    padding='max_length')

    return tok
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [12]:
# in Google collab
# df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/fine_tunnig/df_text_clean.pkl')
df = pd.read_pickle('data/df_text_clean.pkl')
df = df.sample(n=10000)
df = df[['title','label']]
df = df.dropna()


df = df.sample(frac = 1)
df = df.reset_index(drop = True)
df['label'] = df['label'].astype('int')
df.rename(columns={'label': 'labels',
                   "title": "text"},
          inplace=True)

df = df[['text','labels']]


df = df.reset_index(drop = True)
df_train = df.sample(frac = 0.75)
df_test = df.drop(df_train.index)
df_train['labels'] = df_train['labels'].astype('int')
df_test['labels'] = df_test['labels'].astype('int')
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop = True)
df_train.to_csv('train_data.csv',index = False)
df_test.to_csv('test_data.csv', index = False)


dataset = load_dataset('csv', data_files={'train': 'train_data.csv',
                                          'test': 'test_data.csv'},
                       encoding = "ISO-8859-1")

tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment",
                                                         num_labels=4,
                                                         ignore_mismatched_sizes=True)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/oscar/.cache/huggingface/datasets/csv/default-8b22b09cf07f4e8c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/models_sentiment_emotion_bru/models/emotion_model')
dataset = dataset.map(tokenize_data, batched=True)
remove_columns = ['text']
dataset = dataset.map(remove_columns=remove_columns)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [14]:
training_args = TrainingArguments("test_trainer",
                                  num_train_epochs=5)
# model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models_sentiment_emotion_bru/models/emotion_model", num_labels=5, ignore_mismatched_sizes=True)

a = int(len(df_train)*0.75)
train_dataset = dataset['train'].shuffle(seed=10).select(range(0,a))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(a,len(df_train)))
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=eval_dataset)
trainer.train()

Loading cached shuffled indices for dataset at /home/oscar/.cache/huggingface/datasets/csv/default-8b22b09cf07f4e8c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-1474a42298ab6129.arrow


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.93 GiB total capacity; 4.93 GiB already allocated; 36.94 MiB free; 4.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
metric = load_metric("precision")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    
)
trainer.evaluate()

In [ ]:
save_directory = 'model/fine_tunnig'
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

In [ ]:
 
model_name = "model/fine_tunnig"
model_text = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer_sentiment = AutoTokenizer.from_pretrained(model_name)
classifier_sentiment = pipeline("text-classification", model=model_text, tokenizer=tokenizer_sentiment)


In [ ]:
text="asdnkflasñdf sdkmgsdf g"
result = classifier_sentiment(text)
result 

In [ ]:
FN=0
TN=0
TP=0
FP=0
for i in range (0,len( df_test)):
    text = df_test['text'][i]
    result = classifier_sentiment(text)
    if result[0]['label'] == 'LABEL_1' and df_test['labels'][i]== 1:
        TP +=1
    if result[0]['label'] == 'LABEL_1' and df_test['labels'][i]== 0:
        FP +=1
    if result[0]['label'] == 'LABEL_0' and df_test['labels'][i]== 0:
        TN +=1
    if result[0]['label'] == 'LABEL_0' and df_test['labels'][i]== 1:
        FN +=1
acc = (TP+TN)/(TP+TN+FP+FN)
precision = TP /(TP + FP)
recall = TP /(TP +FN)

In [ ]:
acc = (TP+TN)/(TP+TN+FP+FN)
precision = TP /(TP + FP)
recall = TP /(TP + FN)
print("accuracy: ",acc)
print("precision:", precision)
print("recall:", recall)